In [20]:
import os
import numpy as np
import torch
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data, InMemoryDataset
from pathlib import Path
from ts_reader import collect_ts_flat_labels, get_class_weights

In [17]:
class GraphDataset(InMemoryDataset):
    """
    PyG dataset for patient graphs (Load precomputed graphs from .pt files).
    """
    def __init__(self, config, transform=None, pre_transform=None):
        super(GraphDataset, self).__init__(root=config["data_dir"], transform=transform, pre_transform=pre_transform)

        # === 1.load the saved graph ===
        graph_path = Path(config["graph_dir"]) / f"diagnosis_graph_{config['mode']}_k{config['k']}.pt"
        print(f"==> Loading precomputed graph from {graph_path}")
        self.graph_data = torch.load(graph_path,weights_only=False)

        # === 2. read the seq, falt, label ===
        seq, flat, labels, info, N, _, _ = collect_ts_flat_labels(
            config["data_dir"], config["ts_mask"], config["task"], config["add_diag"], debug=config["debug"]
        )
        self.info = info

        # === 3. process edge attr ===
        x = torch.tensor(seq, dtype=torch.float)  # 时间序列数据
        flat = torch.tensor(flat, dtype=torch.float)  # 静态特征
        y = torch.tensor(labels, dtype=torch.long if config["task"] == "ihm" else torch.float)  # 目标标签

        # === load `edge_index` and `edge_attr` ===
        edge_index = self.graph_data.edge_index
        edge_attr = self.graph_data.edge_attr

        # === generate PyG `Data` 对象 ===
        data = Data(x=x, edge_index=edge_index, y=y, flat=flat, edge_attr=edge_attr, **define_masks(N))

        self.data = data
        
        self.class_weights = get_class_weights(labels) if config["task"] == "ihm" else None

    def __repr__(self):
        return f"GraphDataset(num_nodes={self.data.num_nodes}, num_edges={self.data.num_edges})"


# === output train/val/test masks ===
def define_masks(N, train_ratio=0.7, val_ratio=0.15):
    """
    Create PyG node masks for train/val/test split.
    """
    train_n = int(N * train_ratio)
    val_n = int(N * val_ratio)
    idx_train = range(train_n)
    idx_val = range(train_n, train_n + val_n)
    idx_test = range(train_n + val_n, N)

    masks = {
        "train_mask": torch.zeros(N, dtype=torch.bool),
        "val_mask": torch.zeros(N, dtype=torch.bool),
        "test_mask": torch.zeros(N, dtype=torch.bool),
    }
    masks["train_mask"][list(idx_train)] = True
    masks["val_mask"][list(idx_val)] = True
    masks["test_mask"][list(idx_test)] = True

    return masks

# ===generate DataLoader ===
def get_dataloader(config, batch_size=32, shuffle=True):
    """
    Create PyG dataloader for training.
    """
    dataset = GraphDataset(config)
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)


In [ ]:
config = {
    "data_dir": "/home/mei/nas/docker/thesis/data/mmap",  # mmap 存储路径
    "graph_dir": "/home/mei/nas/docker/thesis/data/graphs",  # 你的 `.pt` 图存储路径
    "mode": "k_closest",
    "k": 3,
    "ts_mask": True,
    "task": "ihm",
    "add_diag": False,
    "debug": False
}



In [63]:
train_loader = get_dataloader(config, batch_size=64)

for batch in train_loader:
    print(batch)

==> Loading precomputed graph from /home/mei/nas/docker/thesis/data/graphs/diagnosis_graph_k_closest_k3.pt
DataBatch(x=[9451, 290, 115], edge_index=[2, 28353], edge_attr=[28353], y=[9451], flat=[9451, 104], train_mask=[9451], val_mask=[9451], test_mask=[9451], batch=[9451], ptr=[2])


In [27]:
dataset = GraphDataset(config)
print(dataset)

==> Loading precomputed graph from /home/mei/nas/docker/thesis/data/graphs/diagnosis_graph_k_closest_k3.pt
GraphDataset(num_nodes=9451, num_edges=28353)


In [62]:
from torch_geometric.loader import NeighborLoader

In [ ]:

train_loader = NeighborLoader(
    dataset.data,  # 这里的 dataset.data 是完整的 PyG Data
    num_neighbors=[10, 10],  # 每层邻居采样数
    batch_size=64,  # mini-batch 大小
    input_nodes=dataset.data.train_mask,  # 仅从 train_mask 选取数据
    shuffle=True,
)

for batch in train_loader:
    print(batch)
    break 